### Labb kod - för att spara till Datalake

In [ ]:
# from pathlib import Path
from s3pathlib import S3Path
import jsonargparse
import requests
from loguru import logger

import log_utils

LINK_TO_XML_FILE = {
    "mit": "https://news.mit.edu/rss/topic/artificial-intelligence2",
}
path = S3Path(
    "s3://data.lake/",
    endpoint_url="http://127.0.0.1:9000",
    access_key="save.load",
    secret_key="save.load89"
)


def get_metadata_info(blog_name: str) -> str:
    assert (
        blog_name in LINK_TO_XML_FILE
    ), f"{blog_name=} not supported. Supported blogs: {list(LINK_TO_XML_FILE)}"
    blog_url = LINK_TO_XML_FILE[blog_name]
    response = requests.get(blog_url)
    xml_text = response.text
    # xml_text = response.content.decode("utf-8")
    return xml_text


def save_metadata_info(xml_text: str, blog_name: str) -> None:
    # path_xml_dir = Path("data/data_lake") / blog_name
    path_xml_dir = path / blog_name
    path_xml_dir.mkdir(exist_ok=True, parents=True)
    # with open(path_xml_dir / "metadata.xml", "w") as f:
    #     f.write(xml_text)
    # with open(path_xml_dir / "metadata.xml", "w", encoding="utf-8") as f:
    #     f.write(xml_text)
    s3_file = path / blog_name / "metadata.xml"
    s3_file.write_text(xml_text)

def main(blog_name: str) -> None:
    logger.info(f"Processing {blog_name}")
    xml_text = get_metadata_info(blog_name)
    save_metadata_info(xml_text, blog_name)
    logger.info(f"Done processing {blog_name}")


def parse_args() -> jsonargparse.Namespace:
    parser = jsonargparse.ArgumentParser()
    parser.add_function_arguments(main)
    return parser.parse_args()


# if __name__ == "__main__":
#     args = parse_args()
#     log_utils.configure_logger(log_level="DEBUG")
#     main(**args)


In [23]:
blog_name = "mit"
mit_xml_file = get_metadata_info(blog_name)
save_metadata_info(mit_xml_file, blog_name)

### Labb kod för att hämta från datalake

In [ ]:
import uuid
from datetime import datetime
from pathlib import Path

import jsonargparse
import pandas as pd
from bs4 import BeautifulSoup
from loguru import logger
from s3pathlib import S3Path


from newsfeed import log_utils
from newsfeed.datatypes import BlogInfo

path = S3Path(
    "s3://data.lake/",
    endpoint_url="http://127.0.0.1:9000",
    access_key="save.load",
    secret_key="save.load89"
)



def create_uuid_from_string(val: str) -> str:
    assert isinstance(val, str)
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, val))


def load_metadata(blog_name: str) -> BeautifulSoup:
    # metadata_path = path / blog_name / "metadata.xml"
    # # with open(metadata_path) as f:
    # #     xml_text = f.read()
    s3_file = path / blog_name / "metadata.xml"
    xml_text = s3_file.read_text()
    

    parsed_xml = BeautifulSoup(xml_text, "xml")
    return print("Metadata loaded and parsed"), parsed_xml


def extract_articles_from_xml(parsed_xml: BeautifulSoup) -> list[BlogInfo]:
    articles = []
    for item in parsed_xml.find_all("item"):
        raw_blog_text = item.find("content:encoded").text
        soup = BeautifulSoup(raw_blog_text, "html.parser")
        blog_text = soup.get_text()
        title = item.title.text
        unique_id = create_uuid_from_string(title)
        article_info = BlogInfo(
            unique_id=unique_id,
            title=title,
            description=item.description.text,
            link=item.link.text,
            blog_text=blog_text,
            published=pd.to_datetime(item.pubDate.text).date(),
            timestamp=datetime.now(),
        )
        articles.append(article_info)

    return articles


def save_articles(articles: list[BlogInfo], blog_name: str) -> None:
    save_dir = Path("data/data_warehouse", blog_name, "articles")
    save_dir.mkdir(exist_ok=True, parents=True)
    for article in articles:
        save_path = save_dir / article.filename
        with open(save_path, "w") as f:
            f.write(article.json(indent=2))


def main(blog_name: str) -> None:
    logger.info(f"Processing {blog_name}")
    parsed_xml = load_metadata(blog_name)
    articles = extract_articles_from_xml(parsed_xml)
    save_articles(articles, blog_name)
    logger.info(f"Done processing {blog_name}")


def parse_args() -> jsonargparse.Namespace:
    parser = jsonargparse.ArgumentParser()
    parser.add_function_arguments(main)
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    log_utils.configure_logger(log_level="DEBUG")
    main(**args)


ImportError: cannot import name 'log_utils' from 'newsfeed' (f:\AI-24-programering\Python-programing-Hannes-Fredriksson\.venv\Lib\site-packages\newsfeed\__init__.py)